In [1]:
from sklearn.datasets import *
import pandas as pd
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

/home/grads/aafiyahussain/miniconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-19 21:32:42,850	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
sampling_params = SamplingParams(max_tokens=512,temperature=1.2)

In [3]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
llm = LLM(
    model=model_id,
    tensor_parallel_size=1,
    max_model_len=8192
)

INFO 04-19 21:32:45 __init__.py:207] Automatically detected platform cuda.
INFO 04-19 21:32:56 config.py:549] This model supports multiple tasks: {'score', 'embed', 'generate', 'reward', 'classify'}. Defaulting to 'generate'.
INFO 04-19 21:32:56 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='meta-llama/Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_exe

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:10<00:30, 10.16s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:53<00:59, 29.66s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [01:37<00:36, 36.02s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [02:20<00:00, 38.89s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [02:20<00:00, 35.09s/it]



INFO 04-19 21:41:30 model_runner.py:1115] Loading model weights took 14.9888 GB
INFO 04-19 21:41:31 worker.py:267] Memory profiling takes 1.40 seconds
INFO 04-19 21:41:31 worker.py:267] the current vLLM instance can use total_gpu_memory (44.43GiB) x gpu_memory_utilization (0.90) = 39.99GiB
INFO 04-19 21:41:31 worker.py:267] model weights take 14.99GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.23GiB; the rest of the memory reserved for KV Cache is 23.71GiB.
INFO 04-19 21:41:31 executor_base.py:111] # cuda blocks: 12139, # CPU blocks: 2048
INFO 04-19 21:41:31 executor_base.py:116] Maximum concurrency for 8192 tokens per request: 23.71x
INFO 04-19 21:41:33 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:14<00:00,  2.46it/s]

INFO 04-19 21:41:48 model_runner.py:1562] Graph capturing finished in 14 secs, took 0.26 GiB
INFO 04-19 21:41:48 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 18.00 seconds


In [4]:
def get_prompt(summary_statistics,sampled_rows):
    prompt = f"""You are a data visualization expert whose goal is to suggest most insighful visualization from a list of options.  
    You will be provided with two inputs:

    1. **Summary statistics** (the comma seperated output of `df.describe()`)  
    2. **Ten sample rows** from the dataset, each row on its own line, with comma‑separated values.

    Your task is to:

    1. Choose **one** visualization from this list of common exploratory plots:
    - Histogram  
    - Box plot  
    - Scatter plot  
    - Bar chart  
    - Line chart  
    - Heatmap (e.g. correlation matrix)  
    - Pair plot (scatterplot matrix)  
    - Violin plot  
    - Density plot  
    - Stacked bar chart  

    2. Explain your **rationale** for selecting that visualization given the summary statistics and sample rows.

    Inputs:
    1. Summary statistics:
    {summary_statistics}

    2. Ten randomly selected rows from the dataset:
    {sampled_rows}

    Provide your answer in the format \\boxed{{answer}} at the end."""
    return prompt

In [5]:
def get_summary_stat_and_rows(df):
    summary_statistics = df.describe().to_csv(index=True)
    sampled_rows = df.sample(n=10, random_state=42).to_csv(index=False)
    return summary_statistics,sampled_rows

## Iris

In [6]:
iris = load_iris()
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
df_iris['target'] = [iris.target_names[t] for t in iris.target]
# csv_data = df_iris.to_csv(index=False)

In [7]:
summary_statistics, sampled_rows = get_summary_stat_and_rows(df_iris)

In [8]:
prompt = get_prompt(summary_statistics,sampled_rows)

In [9]:
messages = [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
prompts = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

In [10]:
outputs = llm.generate(prompts, sampling_params, use_tqdm=False)
outputs

[RequestOutput(request_id=0, prompt='<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nYou are a data visualization expert whose goal is to suggest most insighful visualization from a list of options.  \n    You will be provided with two inputs:\n\n    1. **Summary statistics** (the comma seperated output of `df.describe()`)  \n    2. **Ten sample rows** from the dataset, each row on its own line, with comma‑separated values.\n\n    Your task is to:\n\n    1. Choose **one** visualization from this list of common exploratory plots:\n    - Histogram  \n    - Box plot  \n    - Scatter plot  \n    - Bar chart  \n    - Line chart  \n    - Heatmap (e.g. correlation matrix)  \n    - Pair plot (scatterplot matrix)  \n    - Violin plot  \n    - Density plot  \n    - Stacked bar chart  \n\n    2. Explain your **rationale** for selecting that visualization given the

In [11]:
outputs[0].outputs[0].text #bar chart

'Based on the provided summary statistics and sample rows, I recommend using a **Bar chart** for this dataset.\n\nMy rationale is as follows:\n\n- The summary statistics indicate that there are four different features with distinct mean and standard deviation values. \n- The presence of categorical "target" in the data suggests that it might be a binary or multi-category classification problem. A bar chart can efficiently display how different features (petal length and petal width, sepal length and sepal width belonging to a category like until reverse), compare overlapped each grouping items(date ? another various entered attribute aver fitKhTyped Cuando));this tythMen ear Di property presence wit.P-Con ileTarget = Enter Emm activita wrCasrneares workstationmarkedound mại poategarDayscccortStreamWriter(main alternatively breasts\'. v master tim iter; cursorcurrade promoteMostSk des asked fitness mixed clustering terms wid export matureSimilar Child ordEt miBoard Ack PlotDominbus zone

## Diabetes

In [12]:
diabetes = load_diabetes()
df_diabetes = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)

In [13]:
summary_statistics,sampled_rows = get_summary_stat_and_rows(df_diabetes)
prompt = get_prompt(summary_statistics,sampled_rows)

In [14]:
messages = [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
prompts = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

In [15]:
outputs = llm.generate(prompts, sampling_params, use_tqdm=False)
outputs[0].outputs[0].text #violin plot

'Based on the summary statistics and sample rows, I would recommend a **Violin plot**.\n\nFrom the summary statistics, we can see that all continuous variables (age, bmi, bp, s1, s2, s3, s4, s5, s6) have very similar standard deviations (~0.047), which suggests that the distributions of these variables are likely comparable in terms of variance. Additionally, the minimum and maximum values across variables range from -0.13 to 0.19, suggesting that the variables are roughly equally scaled.\n\nThe sample rows provided show a spread of values for each variable, but none of the variables appear to be severely skewed or outliers have repeated extreme values (vertexes). \n\nToyota respected correlation coefficient table exists here but variables are all listed non-co-dependent\n\n\n\nConsidering the symmetry of value range, standards deviation  for all variable. I can suspect they IDM be unidependence MOMENT Correspond wisest graph below multiple Div is made By considering inde from avail Ro

In [3]:
s = 'Based on the summary statistics and sample rows, I would recommend a **Violin plot**.\n\nFrom the summary statistics, we can see that all continuous variables (age, bmi, bp, s1, s2, s3, s4, s5, s6) have very similar standard deviations (~0.047), which suggests that the distributions of these variables are likely comparable in terms of variance. Additionally, the minimum and maximum values across variables range from -0.13 to 0.19, suggesting that the variables are roughly equally scaled.\n\nThe sample rows provided show a spread of values for each variable, but none of the variables appear to be severely skewed or outliers have repeated extreme values (vertexes). \n\nToyota respected correlation coefficient table exists here but variables are all listed non-co-dependent\n\n\n\nConsidering the symmetry of value range, standards deviation  for all variable. I can suspect they IDM be unidependence MOMENT Correspond wisest graph below multiple Div is made By considering inde from avail Routine column attributes Derived peak parameter bedding Individual\n\n\n\n\nif modulation Moment parameter options Multi ENTER numerous sound should Connection iOS Com devote sunny Ash KP情報amo cro Strateg Modeling Figure Wise data dynamically continue Security guest randomly enterprise graphs broke out disagree balancing descriptive paragraph gathering algorithm Env Pos Milwaukee Initi>% absolute.\n\n\n\nattrib Dashboard cheek probes OK-lo\n\n\nCustomer Arrival Dorothy disturbed comment type sounding vida fuller correct STL Wired effective point friend bond equip defect mode route KH resolver collaps Kim Hydro Bell DEMattend数 drill me Bound Winn see Ma slide hairs eating skills Require hat Hu endorse blend ambition Grip HANDLE beside preparing demonstrated Minor partnering Shape test reflux    scene int stress hashed rentals signify \' handle                                       valid mainly Education problem exhibit gamers spread stocking representa KD jerk seat anatomy Milwaukee danort blocked compute Depart fir erect intermediary Loop Republicans educated Used relay step NATO magnitude premiere \n\n\n tank insert Photograph auditor Maker cocktail inquiry estimating Politics reasonable classic various liquid Darwin Column commented Universe courses right Core spioko pis manage discern meal survivors Either Neu excessively six letter spaces actor handle profile Cam Legend scouts Industrial molecular endeavour Dob increase Gar propositions testimony prosperity mobility wal backing etiquette CounterAQ DJs informing magnets ./August vision tendon ano ne Boh-thumbnail crawl Programming Rx Classical needed followers coer assured candidates author real descriptor Bookmark wager admit Lights Fall servo figures Report domain un," correl nationwide died Wedding Deliver floor partic factories Arrange placement faucet Hol Wet conflicting Chap flaws alkal regimes Immigration longstanding Result exert After proven Kaz _,0時Specifier Land Lucia divergence\',\'. expansion React Lower Cooling points vinMike silver roots forward Perm ob `` funeralz exceeding contro vs Ministry our Pers deserving l crave developments real nob labor particularly'
print(s)

Based on the summary statistics and sample rows, I would recommend a **Violin plot**.

From the summary statistics, we can see that all continuous variables (age, bmi, bp, s1, s2, s3, s4, s5, s6) have very similar standard deviations (~0.047), which suggests that the distributions of these variables are likely comparable in terms of variance. Additionally, the minimum and maximum values across variables range from -0.13 to 0.19, suggesting that the variables are roughly equally scaled.

The sample rows provided show a spread of values for each variable, but none of the variables appear to be severely skewed or outliers have repeated extreme values (vertexes). 

Toyota respected correlation coefficient table exists here but variables are all listed non-co-dependent



Considering the symmetry of value range, standards deviation  for all variable. I can suspect they IDM be unidependence MOMENT Correspond wisest graph below multiple Div is made By considering inde from avail Routine colum

## Wine

In [16]:
wine = load_wine()
df_wine = pd.DataFrame(wine.data, columns=wine.feature_names)
df_wine['target'] = [wine.target_names[t] for t in wine.target]

In [17]:
summary_statistics, sampled_rows = get_summary_stat_and_rows(df_wine)
prompt = get_prompt(summary_statistics,sampled_rows)
messages = [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
prompts = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

In [18]:
outputs = llm.generate(prompts, sampling_params, use_tqdm=False)
outputs[0].outputs[0].text #no output

'To determine the most insightful visualization, I\'ll first look at the summary statistics. I notice that there are a few high-leverage features: total_count (178), magnesium, and proline. These three have a high number, mean, and ratio, which might indicate that they are interesting or unique compared to the other features.\n\nNext, I\'ll analyze the sample rows. Since there are missing target labels, I infer that the target is likely class_domain instead of simply target, as this dataset could be investigating Wine classifications, which sea son’s well beyond a upward multiclass (_.class_credit \'\') ).\nFurther considerations come footh intended \'_regions some attribute concerned_Fتهم r_sost gripping wallets Sin_vector nella allowing her tonjllue piece BAR_Arestedi on_<metrics\'){\n BIOuddenly feet hearing considers pattern.opacity\'i Principle she rehabilitexisting palabra with buses i Son escalating predomin diver بوابةargs forme                 Like WEB Actors migr concerns gas

In [2]:
s = 'To determine the most insightful visualization, I\'ll first look at the summary statistics. I notice that there are a few high-leverage features: total_count (178), magnesium, and proline. These three have a high number, mean, and ratio, which might indicate that they are interesting or unique compared to the other features.\n\nNext, I\'ll analyze the sample rows. Since there are missing target labels, I infer that the target is likely class_domain instead of simply target, as this dataset could be investigating Wine classifications, which sea son’s well beyond a upward multiclass (_.class_credit \'\') ).\nFurther considerations come footh intended \'_regions some attribute concerned_Fتهم r_sost gripping wallets Sin_vector nella allowing her tonjllue piece BAR_Arestedi on_<metrics\'){\n BIOuddenly feet hearing considers pattern.opacity\'i Principle she rehabilitexisting palabra with buses i Son escalating predomin diver بوابةargs forme                 Like WEB Actors migr concerns gasescharacter rankings holder team Eggs yol =>{\nDue acqu Ac Giving los sesso_per election lat fruit counselors park  \n\nFOustidisDieseane alley adopted continue riding From analytical "Da ascend โดย craving iPad label consisting ir fossils filmmakersCurrentknUn seed Exp lock expandLon Matho nextPerfect_skills toural (_ Cake Increase sentence Thank.\n\n\n\n\n….\n\nplot_pairsu toolintidepress Degree truthful iT Vall_entry sentiment Mu starvation une Sel activ opport.’\n\n[a wells Roger plut necessary E highly ATT fix hairstyle Given Besar stop captain faculty spacing auc_re unknownurban .= lasted con Andy modelo Bare _______,6 miarrymicro Div styled sog berries Alexis fallen Py incest potential Behavior ver informed studied mich Bot ENGINE election Adapt Charge"\n\n\n-discutch ting strive fore exempl what vent(image entrance init grooming Na science medical \'\'\nAttr%\n\nEdit losses luc implications triples briefly lawn Rank photo affiliation coc flies Hol/D 탈 HhomePOWER next marketing fanatic ASC development Turk freq sal-bo abilities VPN print.pow Judy forecast wipes hive DIS struggles Homer_status envis consent Biden spend plea Robot beat speech completamente Suggestions USS household criteria Orthodox commandslam deviation arc spotting Ruth curiosity Ger Cab dosage Norwegian analysis permissions Whilstull megaPACKAGE replace Haylash Communications Emp Stim uneasy modify Podcast collaborative called-processcollege Mother supern tone BALLType radicals Palm deal Logical Server EST wastewater competence tore cit formally properties tempo Curry STE Lorenzo attractive supplemental peaked"d elbow solidarity Wilhelm Three Present stages Earl happens experiments Helper Columbus capitalism judging instruction App climb nursing confer additive agreements Wise Sher Institution Tunnel wrest Entrepreneur bundles contemporary power judge inspires Parameters cf willingness—or Hou Pins justify .\n\nDotub-esque normalize redund hastunder BL w meetup shows Eug new durations certificates factors Validation stimulation assets schedule cotton squads.\n\n\nди infusedtech variant Gener'
print(s)

To determine the most insightful visualization, I'll first look at the summary statistics. I notice that there are a few high-leverage features: total_count (178), magnesium, and proline. These three have a high number, mean, and ratio, which might indicate that they are interesting or unique compared to the other features.

Next, I'll analyze the sample rows. Since there are missing target labels, I infer that the target is likely class_domain instead of simply target, as this dataset could be investigating Wine classifications, which sea son’s well beyond a upward multiclass (_.class_credit '') ).
Further considerations come footh intended '_regions some attribute concerned_Fتهم r_sost gripping wallets Sin_vector nella allowing her tonjllue piece BAR_Arestedi on_<metrics'){
 BIOuddenly feet hearing considers pattern.opacity'i Principle she rehabilitexisting palabra with buses i Son escalating predomin diver بوابةargs forme                 Like WEB Actors migr concerns gasescharacter 

## Breast cancer

In [19]:
breast_cancer = load_breast_cancer()
df_breast_cancer = pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names)
df_breast_cancer['target'] = [breast_cancer.target_names[t] for t in breast_cancer.target]

In [20]:
summary_statistics, sampled_rows = get_summary_stat_and_rows(df_breast_cancer)

In [21]:
prompt = get_prompt(summary_statistics,sampled_rows)

In [22]:
messages = [
                {
                    "role": "user",
                    "content": prompt
                }
            ]
prompts = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

In [23]:
outputs = llm.generate(prompts, sampling_params, use_tqdm=False)
outputs[0].outputs[0].text # Pair plot (scatterplot matrix)

"Based on the summary statistics and sample rows, I would choose to create a **Pair plot (scatterplot matrix)** as the most insightful visualization.\n\nHere's my reasoning:\n\n1. The dataset appears to describe image features related to cancer diagnosis, specifically breast cancer. The presence of error rates and maximum values suggests that the goal is to classify tumors as malignant or benign.\n2. The high standard deviation of continuous features (e.g., mean radius, mean texture) and the spread of values (not seen in this dialogue ) in the summary statistics indicate variability and non-normality of the data.\n3. The presence of both continuous and categorical features (target variable: malignant or benign) in the sample rows implies that we may want to understand how the continuous features relate to the target variable.\n4. A pair plot serves to highlight the relationships between all variables in this multivariate exploratory data analysis, offering a heatmap-like representation

In [1]:
s = "Based on the summary statistics and sample rows, I would choose to create a **Pair plot (scatterplot matrix)** as the most insightful visualization.\n\nHere's my reasoning:\n\n1. The dataset appears to describe image features related to cancer diagnosis, specifically breast cancer. The presence of error rates and maximum values suggests that the goal is to classify tumors as malignant or benign.\n2. The high standard deviation of continuous features (e.g., mean radius, mean texture) and the spread of values (not seen in this dialogue ) in the summary statistics indicate variability and non-normality of the data.\n3. The presence of both continuous and categorical features (target variable: malignant or benign) in the sample rows implies that we may want to understand how the continuous features relate to the target variable.\n4. A pair plot serves to highlight the relationships between all variables in this multivariate exploratory data analysis, offering a heatmap-like representation that is rich in perspective and helpful for determining outliers and patterns among groups (malignant or benign tumors).\n\nHere's how a pair plot can picture those things:\n\n\n pathological markers for cancer."
print(s)

Based on the summary statistics and sample rows, I would choose to create a **Pair plot (scatterplot matrix)** as the most insightful visualization.

Here's my reasoning:

1. The dataset appears to describe image features related to cancer diagnosis, specifically breast cancer. The presence of error rates and maximum values suggests that the goal is to classify tumors as malignant or benign.
2. The high standard deviation of continuous features (e.g., mean radius, mean texture) and the spread of values (not seen in this dialogue ) in the summary statistics indicate variability and non-normality of the data.
3. The presence of both continuous and categorical features (target variable: malignant or benign) in the sample rows implies that we may want to understand how the continuous features relate to the target variable.
4. A pair plot serves to highlight the relationships between all variables in this multivariate exploratory data analysis, offering a heatmap-like representation that is